In [ ]:
# inference_densenet_soil.ipynb

import os
import numpy as np
from PIL import Image
import torch
from torchvision import models, transforms
import torch.nn as nn

# --- Config ---
IMAGE_SIZE = 224
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
MODEL_PATH = 'densenet121_best_minf1.pth'
LABELS_PATH = 'label_classes.npy'

# --- Load label classes ---
label_classes = np.load(LABELS_PATH, allow_pickle=True)
num_classes = len(label_classes)

# --- Image preprocessing ---
transform = transforms.Compose([
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
])

# --- Model setup ---
model = models.densenet121(pretrained=False)
model.classifier = nn.Sequential(
    nn.Linear(model.classifier.in_features, 256),
    nn.ReLU(),
    nn.Dropout(0.3),
    nn.Linear(256, num_classes)
)
model.load_state_dict(torch.load(MODEL_PATH, map_location=DEVICE))
model = model.to(DEVICE)
model.eval()

def predict_image(image_path):
    image = Image.open(image_path).convert('RGB')
    image = transform(image).unsqueeze(0).to(DEVICE)  # Add batch dimension
    with torch.no_grad():
        outputs = model(image)
        probs = torch.softmax(outputs, dim=1).cpu().numpy()[0]
        pred_idx = probs.argmax()
        pred_label = label_classes[pred_idx]
    return pred_label, probs

# --- Example usage ---
test_image_path = '/kaggle/input/soil-classification/soil_classification-2025/test/example.jpg'  # replace with your image path

pred_label, pred_probs = predict_image(test_image_path)
print(f"Predicted soil type: {pred_label}")
print(f"Class probabilities: {pred_probs}")